In [1]:
# ================================
# Grazioso Salvare Rescue Dashboard
# CS-340 Project Two (CSV Version)
# Developed by Ayomide Akingbemisilu
# ================================

from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
import os
import glob

# -------------------------------
# 1) FIND + LOAD CSV (AUTO-DETECT)
# -------------------------------
csv_candidates = [
    "aac_shelter_outcomes.csv",
    os.path.join("code_files", "aac_shelter_outcomes.csv"),
    os.path.join("..", "code_files", "aac_shelter_outcomes.csv"),
]

csv_path = None
for p in csv_candidates:
    if os.path.exists(p):
        csv_path = p
        break

if not csv_path:
    raise FileNotFoundError(
        "Could not find aac_shelter_outcomes.csv in ., code_files/, or ../code_files/."
    )

df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()
print("✅ CSV loaded from:", csv_path)

has_map_cols = {"location_lat", "location_long"}.issubset(df.columns)

# -------------------------------
# 2) FIND + LOAD LOGO (AUTO-DETECT)
#    (works whether cwd is workspace OR code_files)
# -------------------------------
logo_encoded = ""
logo_path = None

logo_candidates = [
    "Grazioso Salvare Logo.png",
    "Grazioso Salvare Logo.PNG",
    os.path.join("code_files", "Grazioso Salvare Logo.png"),
    os.path.join("code_files", "Grazioso Salvare Logo.PNG"),
    os.path.join("..", "code_files", "Grazioso Salvare Logo.png"),
    os.path.join("..", "code_files", "Grazioso Salvare Logo.PNG"),
]

for p in logo_candidates:
    if os.path.exists(p):
        logo_path = p
        break

# wildcard fallback (search a couple likely places)
if not logo_path:
    patterns = [
        os.path.join(".", "*Logo*.png"),
        os.path.join("code_files", "*Logo*.png"),
        os.path.join("..", "code_files", "*Logo*.png"),
        os.path.join(".", "*Logo*.PNG"),
        os.path.join("code_files", "*Logo*.PNG"),
        os.path.join("..", "code_files", "*Logo*.PNG"),
    ]
    for pat in patterns:
        matches = glob.glob(pat)
        if matches:
            logo_path = matches[0]
            break

if logo_path and os.path.exists(logo_path):
    with open(logo_path, "rb") as f:
        logo_encoded = base64.b64encode(f.read()).decode("utf-8")
    print("✅ Logo loaded from:", logo_path)
else:
    print("⚠️ Logo not found. (Checked ., code_files/, and ../code_files/)")

# -------------------------------
# 3) FILTER LOGIC
# -------------------------------
def apply_rescue_filter(dataframe, filter_value):
    dff = dataframe.copy()

    if "age_upon_outcome_in_weeks" in dff.columns:
        dff["age_upon_outcome_in_weeks"] = pd.to_numeric(dff["age_upon_outcome_in_weeks"], errors="coerce")
        dff = dff[dff["age_upon_outcome_in_weeks"].isna() | (dff["age_upon_outcome_in_weeks"] <= 104)]

    if "animal_type" in dff.columns:
        dff = dff[dff["animal_type"].astype(str).str.lower() == "dog"]

    if filter_value == "reset":
        return dff

    if "breed" not in dff.columns:
        return dff

    breed_series = dff["breed"].astype(str)

    if filter_value == "water":
        keywords = "labrador|chesapeake|newfoundland|poodle"
        return dff[breed_series.str.contains(keywords, case=False, na=False)]

    if filter_value == "mountain":
        keywords = "german shepherd|malamute|siberian husky|husky|rottweiler"
        return dff[breed_series.str.contains(keywords, case=False, na=False)]

    if filter_value == "disaster":
        keywords = "doberman|belgian malinois|bloodhound|boxer|collie"
        return dff[breed_series.str.contains(keywords, case=False, na=False)]

    return dff

# -------------------------------
# 4) DASH APP + LAYOUT
# -------------------------------
app = JupyterDash(__name__)

app.layout = html.Div([

    html.Center([
        html.Img(
            src=f"data:image/png;base64,{logo_encoded}" if logo_encoded else "",
            style={"height": "120px", "marginTop": "10px"}
        ),
        html.H2("Grazioso Salvare Rescue Dashboard"),
        html.H4("Developed by Ayomide Akingbemisilu"),
    ]),

    html.Hr(),

    dcc.RadioItems(
        id="rescue-filter",
        options=[
            {"label": "Reset (All)", "value": "reset"},
            {"label": "Water Rescue", "value": "water"},
            {"label": "Mountain / Wilderness Rescue", "value": "mountain"},
            {"label": "Disaster / Individual Tracking", "value": "disaster"},
        ],
        value="reset",
        labelStyle={"display": "block"}
    ),

    html.Hr(),

    dash_table.DataTable(
        id="datatable",
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "minWidth": "90px", "width": "120px", "maxWidth": "240px"}
    ),

    html.Hr(),

    html.H3("Animal Locations"),
    dcc.Graph(id="map"),

    html.Hr(),

    html.H3("Top 10 Breeds (Current Filter)"),
    dcc.Graph(id="breed-chart")

])

# -------------------------------
# 5) CALLBACK
# -------------------------------
@app.callback(
    [Output("datatable", "data"),
     Output("map", "figure"),
     Output("breed-chart", "figure")],
    [Input("rescue-filter", "value")]
)
def update_dashboard(filter_value):

    dff = apply_rescue_filter(df, filter_value)

    if has_map_cols and not dff.empty:
        dff_map = dff.dropna(subset=["location_lat", "location_long"])
        if not dff_map.empty:
            map_fig = px.scatter_mapbox(
                dff_map,
                lat="location_lat",
                lon="location_long",
                hover_name="breed" if "breed" in dff_map.columns else None,
                zoom=9,
                title="Animal Locations"
            )
            map_fig.update_layout(mapbox_style="open-street-map")
        else:
            map_fig = px.scatter_mapbox(title="Animal Locations (No valid coordinates)")
            map_fig.update_layout(mapbox_style="open-street-map")
    else:
        map_fig = px.scatter_mapbox(title="Animal Locations (No data or missing lat/long)")
        map_fig.update_layout(mapbox_style="open-street-map")

    if not dff.empty and "breed" in dff.columns:
        top = dff["breed"].astype(str).value_counts().nlargest(10).reset_index()
        top.columns = ["breed", "count"]
        pie_fig = px.pie(top, names="breed", values="count", title="Top 10 Breeds (Current Filter)")
    else:
        pie_fig = px.pie(title="Top 10 Breeds (No data)")

    return dff.to_dict("records"), map_fig, pie_fig

# -------------------------------
# 6) RUN
# -------------------------------
app.run_server(mode="jupyterlab", debug=True)


✅ CSV loaded from: aac_shelter_outcomes.csv
✅ Logo loaded from: Grazioso Salvare Logo.png


In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import base64
import os
import glob

# IMPORTANT: this is what makes Codio proxy links work
JupyterDash.infer_jupyter_proxy_config()

# ---------- find files (works whether you're in /code_files or not) ----------
def find_file(filename):
    # try current folder
    if os.path.exists(filename):
        return filename
    # try code_files folder
    p = os.path.join("code_files", filename)
    if os.path.exists(p):
        return p
    # last resort: search workspace
    hits = glob.glob(f"**/{filename}", recursive=True)
    return hits[0] if hits else None

CSV_PATH = find_file("aac_shelter_outcomes.csv")
LOGO_PATH = find_file("Grazioso Salvare Logo.png")

if not CSV_PATH:
    raise FileNotFoundError("Could not find aac_shelter_outcomes.csv anywhere in the workspace.")
if not LOGO_PATH:
    raise FileNotFoundError("Could not find Grazioso Salvare Logo.png anywhere in the workspace.")

df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip()

# ---------- encode logo ----------
encoded_logo = base64.b64encode(open(LOGO_PATH, "rb").read()).decode("utf-8")

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Center([
        html.Img(src="data:image/png;base64,{}".format(encoded_logo), style={"height": "140px"}),
        html.H1("Grazioso Salvare Rescue Dashboard"),
        html.H4("Developed by Ayomide Akingbemisilu"),
    ]),
    confirming := html.Hr(),

    dcc.RadioItems(
        id="filter-type",
        options=[
            {"label": "Reset (All)", "value": "all"},
            {"label": "Water Rescue", "value": "water"},
            {"label": "Mountain / Wilderness Rescue", "value": "mountain"},
            {"label": "Disaster / Individual Tracking", "value": "disaster"},
        ],
        value="all",
        inline=False
    ),
    html.Hr(),

    dash_table.DataTable(
        id="datatable",
        columns=[{"name": c, "id": c} for c in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "minWidth": "110px", "width": "140px", "maxWidth": "260px"},
    ),

    html.Hr(),
    html.Div(style={"display": "flex", "gap": "20px"}, children=[
        dcc.Graph(id="map-graph", style={"flex": "1"}),
        dcc.Graph(id="breed-chart", style={"flex": "1"}),
    ])
])

def apply_filter(dff, filter_type):
    # If your CSV doesn’t have the same fields as Mongo, we keep filtering simple + safe
    if filter_type == "all":
        return dff

    # Try best-effort filtering if your CSV has outcome_type / animal_type / breed etc.
    # If not found, it’ll just return unfiltered.
    if "animal_type" in dff.columns:
        if filter_type == "water":
            return dff[dff["animal_type"].astype(str).str.contains("Dog|Labrador|Retriever", case=False, na=False)]
        if filter_type == "mountain":
            return dff[dff["animal_type"].astype(str).str.contains("Dog|Shepherd|Husky", case=False, na=False)]
        if filter_type == "disaster":
            return dff
    return dff

@app.callback(
    Output("datatable", "data"),
    [Input("filter-type", "value")]
)
def update_table(filter_type):
    dff = apply_filter(df, filter_type)
    return dff.to_dict("records")

@app.callback(
    Output("breed-chart", "figure"),
    [Input("datatable", "derived_virtual_data")]
)
def update_pie(view_data):
    dff = pd.DataFrame(view_data) if view_data else df.copy()
    if "breed" not in dff.columns or dff.empty:
        return px.pie(title="Top 10 Breeds (No breed data)")
    top = dff["breed"].astype(str).value_counts().nlargest(10).reset_index()
    top.columns = ["breed", "count"]
    return px.pie(top, names="breed", values="count", title="Top 10 Breeds (Current Filter)")

@app.callback(
    Output("map-graph", "figure"),
    [Input("datatable", "derived_virtual_data")]
)
def update_map(view_data):
    dff = pd.DataFrame(view_data) if view_data else df.copy()

    lat_col = None
    lon_col = None
    for c in dff.columns:
        if c.lower() in ["location_lat", "lat", "latitude"]:
            lat_col = c
        if c.lower() in ["location_long", "lon", "lng", "longitude"]:
            lon_col = c

    if not lat_col or not lon_col or dff.empty:
        return px.scatter_mapbox(title="Animal Locations (No valid lat/long)").update_layout(mapbox_style="open-street-map")

    dff = dff.dropna(subset=[lat_col, lon_col])
    if dff.empty:
        return px.scatter_mapbox(title="Animal Locations (No valid lat/long)").update_layout(mapbox_style="open-street-map")

    fig = px.scatter_mapbox(
        dff,
        lat=lat_col,
        lon=lon_col,
        hover_name="breed" if "breed" in dff.columns else None,
        zoom=9,
        title="Animal Locations"
    )
    fig.update_layout(mapbox_style="open-street-map")
    return fig

# THE IMPORTANT PART: run in external mode so Codio gives you a /proxy/ link
app.run_server(mode="external", host="0.0.0.0", port=8050, debug=False, use_reloader=False)


 * Running on all addresses.
 * Running on http://10.179.87.190:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Dec/2025 17:24:51] "GET /_alive_a4777380-f697-4b81-be66-75846e476c46 HTTP/1.1" 200 -


Dash app running on https://guidemirror-avenuejanet-3000.codio.io/proxy/8050/


127.0.0.1 - - [14/Dec/2025 17:24:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2025 17:24:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2025 17:24:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2025 17:24:57] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2025 17:24:57] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2025 17:24:57] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2025 17:24:57] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2025 17:24:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2025 17:24:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2025 17:24:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2025 17:25:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -